In [24]:
from PIL import Image, ImageDraw
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import math
import random

In [25]:
def sin(a):
    return math.sin(a)
def cos(a):
    return math.cos(a)
def tan(a):
    return math.tan(a)
pi=math.pi
sqrt2=math.sqrt(2)

def deg2rad(t):
    return t/180*pi
def rad2deg(t):
    return t/pi*180

def rotatePoint(p,c,t):
    '''
    rotate a point p=(px,py) about a point c=(cx,cy) by t radians counterclockwise
    '''
    px=p[0]; py=p[1]; px-=c[0]; py-=c[1]
    rx=px*cos(t)-py*sin(t)+c[0]
    ry=px*sin(t)+py*cos(t)+c[1]
    return (rx,ry)

def midpoint(a,b):
    return (a[0]/2+b[0]/2,a[1]/2+b[1]/2)

In [26]:
im=Image.open('bluegradient500x500.jpeg')
draw=ImageDraw.Draw(im)


# Octagon Geometry

Define by center, radius, angle (radians)

**center**= (cx, cy)

**radius**=r

#### When **angle** = 0:

- N point is (cx,cy+r)

- NW point is (cx-r\*sqrt2/2, cy+r\*sqrt2/2)

- W point is (cx-r,cy)

- SW point is (cx-r\*sqrt2/2, cy-r\*sqrt2/2)

- S point is (cx,cy-r)

- SE point is (cx+r\*sqrt2/2, cy-r\*sqrt2/2)

- E point is (cx+r,cy)

- NE point is (cx+r\*sqrt2, cy+r\*sqrt2)

In [27]:
def OctagonPoints(c, r, t):
    '''
    Function for finding vertices of an octagon given: center coordinates c=(cx,cy), radius (r), and angle of rotation (t)
    '''
    cx=c[0]; cy=c[1];
    points=[
        (cx,cy+r),
        (cx-r*sqrt2/2,cy+r*sqrt2/2),
        (cx-r,cy),
        (cx-r*sqrt2/2,cy-r*sqrt2/2),
        (cx,cy-r),
        (cx+r*sqrt2/2,cy-r*sqrt2/2),
        (cx+r,cy),
        (cx+r*sqrt2/2,cy+r*sqrt2/2),
    ]
    for i in range(8):
        points[i]=rotatePoint(points[i],c,t)
    return points

def DrawOctagons(c,r1,r2,t,w,draw):
    '''
    Function for drawing entire board
    '''
    o1=OctagonPoints(c,r1,t); draw.polygon(o1,outline='black',fill='white',width=w)
    o2=OctagonPoints(c,r2,t); draw.polygon(o2,outline='black',fill='white',width=w)
    for i in range(8):
        draw.line([o1[i],o2[i]],fill='black',width=w)

# Chevron Geometry

Defined by center (of square), side length (of square)

**center** = (cx, cy)

**side length** = s

- N point is (cx,cy+s)

- NE point is (cx+s/2, cy+s/2)

- SE point is (cx+s/2, cy-s/2)

- Pocket is (cx, cy)

- SW point is (cx-s/2, cy-s/2)

- NW point is (cx-s/2, cy+s/2)

In [28]:
def ChevronPoints(c,s,t):
    '''
    Function for drawing an individual chevron given: center coordinates c=(cx,cy), side length (s), and angle of rotation (t)
    '''
    cx=c[0]; cy=c[1];
    points=[
        (cx,cy+s),
        (cx+s/2,cy+s/2),
        (cx+s/2,cy-s/2),
        (cx,cy),
        (cx-s/2,cy-s/2),
        (cx-s/2,cy+s/2)
    ]
    for i in range(6):
        points[i]=rotatePoint(points[i],c,t)
    return points

def DrawChevron(c,s,t,draw):
    draw.polygon(ChevronPoints(c,s,t),fill='red')

In [29]:
def CDST(a):
    return a.midpoint1[0]+a.midpoint1[0]+0.00001*(a.midpoint1[1]+a.midpoint2[1])

class CHEV:
    def __init__(self,x1,y1,x2,y2):
        self.midpoint1=(x1,y1)
        self.midpoint2=(x2,y2)
    def __init__(self,m1,m2):
        self.midpoint1=m1
        self.midpoint2=m2
    def pp(self):
        return str(self.midpoint1)+" "+str(self.midpoint2)
class IMG:
    def __init__(self,numValidChevrons,validChevrons, legal1):
        self.chevsInCenter=numValidChevrons
        self.validChevrons=validChevrons
        self.validChevrons.sort(key=CDST)
        self.legal=legal1
    def p(self):
        for i in self.validChevrons:
            print(i.pp())

# Generator

In [30]:
def generateNRandom(n,im,draw,rigged):
    counter=0
    '''
    Image dimensions are 500x500
    rigged is how far the chevron can possibly fall to the octagon center
    '''
    
    minOctagonCenterX=100; maxOctagonCenterX=400
    minOctagonCenterY=100; maxOctagonCenterY=400
    minOctagonLargeRadius=100; maxOctagonLargeRadius=300
    maxRadiusRatio=3; minRadiusRatio=1.3
    minOctagonOutlineWidth=2; maxOctagonOutlineWidth=8
    minChevronSize=20; maxChevronSize=30

    octagonCenterX=random.randrange(minOctagonCenterX,maxOctagonCenterX)
    octagonCenterY=random.randrange(minOctagonCenterY,maxOctagonCenterY)
    minChevronCenterX=octagonCenterX-rigged
    maxChevronCenterX=octagonCenterX+rigged
    minChevronCenterY=octagonCenterY-rigged
    maxChevronCenterY=octagonCenterY+rigged
    octagonLargeRadius=random.randrange(minOctagonLargeRadius,maxOctagonLargeRadius)
    octagonRadiusRatio=random.uniform(minRadiusRatio,maxRadiusRatio)
    octagonSmallRadius=octagonLargeRadius//octagonRadiusRatio
    octagonOutlineWidth=random.randrange(minOctagonOutlineWidth,maxOctagonOutlineWidth)
    octagonAngle=random.uniform(0,2*pi)
    DrawOctagons((octagonCenterX,octagonCenterY),octagonLargeRadius,octagonSmallRadius,octagonAngle,octagonOutlineWidth,draw)
    innerOcto=Polygon(OctagonPoints((octagonCenterX,octagonCenterY),octagonSmallRadius,octagonAngle))
    chevronSize=random.randrange(minChevronSize,maxChevronSize)
    
    chevPolys=[]
    chevsPoints=[]
    chevs=[]

    for i in range(n):
        chevronCenterX=random.randrange(minChevronCenterX,maxChevronCenterX)
        chevronCenterY=random.randrange(minChevronCenterY,maxChevronCenterY)
        chevronAngle=random.uniform(0,2*pi)
        DrawChevron((chevronCenterX,chevronCenterY),chevronSize,chevronAngle,draw)
        chevPoints=ChevronPoints((chevronCenterX,chevronCenterY),chevronSize,chevronAngle)
        thisChevron=CHEV(midpoint(chevPoints[1],chevPoints[2]),midpoint(chevPoints[4],chevPoints[5]))
        chevPolys.append(Polygon(chevPoints))
        chevsPoints.append(chevPoints)
        allinside=True
        for cpt in chevPoints:
            pt=Point(cpt[0],cpt[1])
            if innerOcto.contains(pt):
                continue
            else:
                allinside=False
                break
        if allinside:
            chevs.append(thisChevron)
            counter+=1
    allDistinct=True
    for i in range(n-1):
        for j in range(i,n):
            for cpt in chevsPoints[i]:
                if chevPolys[j].contains(Point(cpt[0],cpt[1])):
                    allDistinct=False
                    break
    

    return IMG(counter,chevs,allDistinct)

In [31]:
def generate0(rigged):
    global im
    img=IMG(0,[],False)
    while ((img.legal==False)or(img.chevsInCenter!=0)):
        im=Image.open('bluegradient500x500.jpeg')
        draw=ImageDraw.Draw(im)
        img=generateNRandom(random.randrange(1,5),im,draw,rigged)
    #im.save("IMG"+id+".jpeg","JPEG")
    return img

def generate1(rigged):
    global im
    img=IMG(0,[],False)
    while ((img.legal==False)or(img.chevsInCenter!=1)):
        im=Image.open('bluegradient500x500.jpeg')
        draw=ImageDraw.Draw(im)
        img=generateNRandom(random.randrange(1,5),im,draw,rigged)
    #im.save("IMG"+id+".jpeg","JPEG")
    return img
def generate2(rigged):
    global im
    img=IMG(0,[],False)
    while ((img.legal==False)or(img.chevsInCenter!=2)):
        im=Image.open('bluegradient500x500.jpeg')
        draw=ImageDraw.Draw(im)
        img=generateNRandom(random.randrange(1,5),im,draw,rigged)
    #im.save("IMG"+id+".jpeg","JPEG")
    return img
def generate3(rigged):
    global im
    img=IMG(0,[],False)
    while ((img.legal==False)or(img.chevsInCenter!=3)):
        im=Image.open('bluegradient500x500.jpeg')
        draw=ImageDraw.Draw(im)
        img=generateNRandom(random.randrange(1,5),im,draw,rigged)
    #im.save("IMG"+id+".jpeg","JPEG")
    return img
def generate4(rigged):
    global im
    img=IMG(0,[],False)
    while ((img.legal==False)or(img.chevsInCenter!=4)):
        im=Image.open('bluegradient500x500.jpeg')
        draw=ImageDraw.Draw(im)
        img=generateNRandom(random.randrange(1,5),im,draw,rigged)
    #im.save("IMG"+id+".jpeg","JPEG")
    return img

In [32]:
id2num={}
im=Image.open('bluegradient500x500.jpeg')
NUMBEROFIMAGES=25
iii=0*NUMBEROFIMAGES
INFO={}
while (iii<NUMBEROFIMAGES*1):
    this=generate0(150)
    id='0'*(5-len(str(iii)))+str(iii)
    if this.legal:
        iii+=1
        INFO[id]=this
        im.save("IMG"+id+".jpeg","JPEG")
        #im.show()

while (iii<NUMBEROFIMAGES*2):
    this=generate1(75)
    id='0'*(5-len(str(iii)))+str(iii)
    if this.legal:
        iii+=1
        INFO[id]=this
        im.save("IMG"+id+".jpeg","JPEG")
        #im.show()


while (iii<NUMBEROFIMAGES*3):
    this=generate2(75)
    id='0'*(5-len(str(iii)))+str(iii)
    if this.legal:
        iii+=1
        INFO[id]=this
        im.save("IMG"+id+".jpeg","JPEG")
       # im.show()


while (iii<NUMBEROFIMAGES*4):
    this=generate3(75)
    id='0'*(5-len(str(iii)))+str(iii)
    if this.legal:
        iii+=1
        INFO[id]=this
        im.save("IMG"+id+".jpeg","JPEG")
      #  im.show()


while (iii<NUMBEROFIMAGES*5):
    this=generate4(75)
    id='0'*(5-len(str(iii)))+str(iii)
    if this.legal:
        iii+=1
        INFO[id]=this
        im.save("IMG"+id+".jpeg","JPEG")
    #    im.show()

In [33]:
for i in INFO:
    print(i+":    ")
    INFO[i].p();

00000:    
00001:    
00002:    
00003:    
00004:    
00005:    
00006:    
00007:    
00008:    
00009:    
00010:    
00011:    
00012:    
00013:    
00014:    
00015:    
00016:    
00017:    
00018:    
00019:    
00020:    
00021:    
00022:    
00023:    
00024:    
00025:    
(206.87279434242924, 155.17346344733426) (223.12720565757076, 166.82653655266574)
00026:    
(119.79395074321583, 269.5707499982643) (136.20604925678418, 288.4292500017357)
00027:    
(401.7528098840871, 281.0261207332543) (420.2471901159129, 290.9738792667457)
00028:    
(301.19598211964075, 123.02488708149086) (302.80401788035925, 148.97511291850913)
00029:    
(324.3392999490886, 346.57455961776463) (305.6607000509114, 339.42544038223537)
00030:    
(270.98827302251624, 352.66539772490125) (289.01172697748376, 361.33460227509875)
00031:    
(157.30832589995228, 60.85453016967205) (170.69167410004772, 83.14546983032795)
00032:    
(233.16871427250237, 230.6872511365923) (252.83128572749763, 223.31274886